In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indian-water-quality-data/water_dataX.csv


In [2]:
!pip install llvmlite --ignore-installed
!pip install pycaret


  Using cached llvmlite-0.39.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.54.1 requires llvmlite<0.38,>=0.37.0rc1, but you have llvmlite 0.39.1 which is incompatible.
  Using cached llvmlite-0.37.0-cp37-cp37m-manylinux2014_x86_64.whl (26.3 MB)
  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.0
    Uninstalling numpy-1.20.0:
      Successfully uninstalled numpy-1.20.0
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.39.1
    Uninstalling llvmlite-0.39.1:
      Successfully uninstalled llvmlite-0.39.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depende

In [3]:
# importing dependencies
import pandas as pd

In [4]:
df = pd.read_csv("../input/indian-water-quality-data/water_dataX.csv", encoding= 'unicode_escape')

In [5]:
df

,STATION CODE,LOCATIONS,STATE,Temp,D.O. (mg/l),PH,CONDUCTIVITY (µmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean,year
0,1393,"DAMANGANGA AT D/S OF MADHUBAN, DAMAN",DAMAN & DIU,30.6,6.7,7.5,203,NAN,0.1,11,27,2014
1,1399,ZUARI AT D/S OF PT. WHERE KUMBARJRIA CANAL JOI...,GOA,29.8,5.7,7.2,189,2,0.2,4953,8391,2014
2,1475,ZUARI AT PANCHAWADI,GOA,29.5,6.3,6.9,179,1.7,0.1,3243,5330,2014
3,3181,RIVER ZUARI AT BORIM BRIDGE,GOA,29.7,5.8,6.9,64,3.8,0.5,5382,8443,2014
4,3182,RIVER ZUARI AT MARCAIM JETTY,GOA,29.5,5.8,7.3,83,1.9,0.4,3428,5500,2014
...,...,...,...,...,...,...,...,...,...,...,...,...
1986,1330,"TAMBIRAPARANI AT ARUMUGANERI, TAMILNADU",NAN,NAN,7.9,738,7.2,2.7,0.518,0.518,202,2003
1987,1450,"PALAR AT VANIYAMBADI WATER SUPPLY HEAD WORK, T...",NAN,29,7.5,585,6.3,2.6,0.155,0.155,315,2003
1988,1403,"GUMTI AT U/S SOUTH TRIPURA,TRIPURA",NAN,28,7.6,98,6.2,1.2,NAN,NAN,570,2003
1989,1404,"GUMTI AT D/S SOUTH TRIPURA, TRIPURA",NAN,28,7.7,91,6.5,1.3,NAN,NAN,562,2003


In [6]:
df = df.replace('NAN',np.nan,regex = True)

In [7]:
#Dropping unnecessary features
df.drop(["STATION CODE", "LOCATIONS","STATE","year"],axis=1,inplace=True)

In [8]:
df.head()

,Temp,D.O. (mg/l),PH,CONDUCTIVITY (µmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean
0,30.6,6.7,7.5,203,NaN,0.1,11,27
1,29.8,5.7,7.2,189,2,0.2,4953,8391
2,29.5,6.3,6.9,179,1.7,0.1,3243,5330
3,29.7,5.8,6.9,64,3.8,0.5,5382,8443
4,29.5,5.8,7.3,83,1.9,0.4,3428,5500


In [9]:
#conversions
df['Temp']=pd.to_numeric(df['Temp'],errors='coerce')
df['D.O. (mg/l)']=pd.to_numeric(df['D.O. (mg/l)'],errors='coerce')
df['PH']=pd.to_numeric(df['PH'],errors='coerce')
df['B.O.D. (mg/l)']=pd.to_numeric(df['B.O.D. (mg/l)'],errors='coerce')
df['CONDUCTIVITY (µmhos/cm)']=pd.to_numeric(df['CONDUCTIVITY (µmhos/cm)'],errors='coerce')
df['NITRATENAN N+ NITRITENANN (mg/l)']=pd.to_numeric(df['NITRATENAN N+ NITRITENANN (mg/l)'],errors='coerce')
df['FECAL COLIFORM (MPN/100ml)']=pd.to_numeric(df['FECAL COLIFORM (MPN/100ml)'],errors='coerce')
df['TOTAL COLIFORM (MPN/100ml)Mean']=pd.to_numeric(df['TOTAL COLIFORM (MPN/100ml)Mean'],errors='coerce')
df['B.O.D. (mg/l)']=pd.to_numeric(df['B.O.D. (mg/l)'],errors='coerce')
df.dtypes

Temp                                float64
D.O. (mg/l)                         float64
PH                                  float64
CONDUCTIVITY (µmhos/cm)             float64
B.O.D. (mg/l)                       float64
NITRATENAN N+ NITRITENANN (mg/l)    float64
FECAL COLIFORM (MPN/100ml)          float64
TOTAL COLIFORM (MPN/100ml)Mean      float64
dtype: object

In [10]:
!pip install numpy==1.20.0

  Using cached numpy-1.20.0-cp37-cp37m-manylinux2010_x86_64.whl (15.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.20.0 which is incompatible.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.20.0 which is incompatible.
tfx-bsl 1.9.0 requires pyarrow<6,>=1, but you have pyarrow 8.0.0 which is incompatible.
tfx-bsl 1.9.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,<3,>=1.15.5, but you have tensor

In [11]:
from pycaret.regression import *
s = setup(df,target = 'B.O.D. (mg/l)',numeric_features=["Temp","D.O. (mg/l)","PH","CONDUCTIVITY (µmhos/cm)",
                           "NITRATENAN N+ NITRITENANN (mg/l)", "FECAL COLIFORM (MPN/100ml)",
                           "TOTAL COLIFORM (MPN/100ml)Mean"])

,Description,Value
0,session_id,3443
1,Target,B.O.D. (mg/l)
2,Original Data,"(1991, 8)"
3,Missing Values,True
4,Numeric Features,7
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1359, 7)"


In [12]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,5.4968,801.6438,24.0237,0.0791,0.6581,1.5619,0.3690
et,Extra Trees Regressor,5.3413,809.9780,24.2064,0.0620,0.6695,1.3619,0.2670
dummy,Dummy Regressor,8.2524,869.0598,25.5220,-0.0656,1.1390,3.6313,0.0090
llar,Lasso Least Angle Regression,8.2524,869.0598,25.5220,-0.0656,1.1390,3.6313,0.0100
omp,Orthogonal Matching Pursuit,7.7865,854.6149,25.3428,-0.1589,1.0549,3.2201,0.0110
gbr,Gradient Boosting Regressor,6.2949,918.8397,26.7092,-0.5619,0.7595,1.9810,0.0720
lightgbm,Light Gradient Boosting Machine,7.3764,857.5475,26.2749,-0.6379,0.8602,2.3502,0.0970
knn,K Neighbors Regressor,6.2406,878.3950,25.9790,-0.6841,0.7573,1.7391,0.0360
xgboost,Extreme Gradient Boosting,6.4492,956.6451,27.6647,-0.7450,0.7221,1.7906,0.1200
catboost,CatBoost Regressor,5.9820,908.7877,27.6798,-1.8909,0.7248,1.8596,1.1960


In [13]:
#checking model hyperparameter
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [17]:
test=pd.DataFrame(columns=
                 ['Temp','D.O. (mg/l)', 'PH', 'CONDUCTIVITY (µmhos/cm)',
                  'NITRATENAN N+ NITRITENANN (mg/l)', 'FECAL COLIFORM (MPN/100ml)',
                  'TOTAL COLIFORM (MPN/100ml)Mean'])

test_input = {'Temp': 29.7,
              'D.O. (mg/l)': 5.8,
              'PH': 6.9,
              'CONDUCTIVITY (µmhos/cm)' :64,
              'NITRATENAN N+ NITRITENANN (mg/l)' :0.5,
              'FECAL COLIFORM (MPN/100ml)' :5382,
              'TOTAL COLIFORM (MPN/100ml)Mean' :8443}
test = test.append(test_input, ignore_index = True)

In [18]:
df_result= predict_model(best,data = test)
df_result.iloc[0]['Label']

2.334270004630089

In [16]:
# saving model
save_model(best,'best_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=['Temp', 'D.O. (mg/l)',
                                                           'PH',
                                                           'CONDUCTIVITY '
                                                           '(µmhos/cm)',
                                                           'NITRATENAN N+ '
                                                           'NITRITENANN (mg/l)',
                                                           'FECAL COLIFORM '
                                                           '(MPN/100ml)',
                                                           'TOTAL COLIFORM '
                                               